In [1]:
! pip install datasets
! pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "

In [2]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
import shutil
import re
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from nltk.corpus import stopwords

stop_words = list(set(stopwords.words('english')))
more_stop_words = ["(", ")", "--", "*", ":", "-", "may", "though", ";",
         "thing", "things", "'d", "'ll", "'m", "'ve", "'t", "'s",
         "'re", "a", "about", "above", "after", "again", "against",
         "ain", "all", "am", "an", "and", "any", "are", "aren't",
         "as", "at", "be", "because", "been", "before", "being", "below",
         "between", "both", "but", "by", "can", "could", "couldn't",
         "did", "didn't", "do", "does", "doesn't", "doing", "don't", "down",
         "during", "each", "few", "for", "from", "further", "had", "hadn't",
         "has", "hasn't", "have", "haven't", "having", "he", "he'd", "he'll",
         "he's", "her", "here", "here's", "hers", "herself", "him", "himself",
         "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in",
         "into", "is", "isn't", "it", "it's", "its", "itself", "just", "let's",
         "me", "mightn't", "more", "most", "mustn't", "my", "myself",
         "needn't", "no", "nor", "not", "now", "o", "of", "off", "on", "once",
         "only", "or", "other", "ought", "our", "ours", "ourselves", "out",
         "over", "own", "same", "shan't", "she", "she'd", "she'll", "she's",
         "should", "should've", "shouldn't", "so", "some", "such", "t", "than",
         "that", "that'll", "that's", "the", "their", "theirs", "them",
         "themselves", "then", "there", "there's", "these", "they", "they'd",
         "they'll", "they're", "they've", "this", "those", "through", "to",
         "too", "under", "until", "up", "very", "was", "wasn't", "we", "we'd",
         "we'll", "we're", "we've", "were", "weren't", "what", "what's",
         "when", "when's", "where", "where's", "which", "while", "who",
         "who's", "whom", "why", "why's", "will", "with", "won't", "would",
         "wouldn", "wouldn't", "y'all", "you", "you'd", "you'll", "you're",
         "you've", "your", "yours", "yourself", "yourselves", ",", ".", "!",
         "?", "'", '"', "I", "i","im","Im","br"]

stop_words.extend(more_stop_words)
stop_words = list(set(stop_words))

In [5]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
cleaned_data_path = '/content/drive/MyDrive/Boston College/2025 Spring/NLP/NLP project/processed_downsampled_data.csv'
dest_path = '/content/data.csv'

In [7]:
shutil.copy(cleaned_data_path, dest_path)

'/content/data.csv'

# Data cleaning

In [8]:
data = pd.read_csv("data.csv")
data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,length,polarity,subjectivity,char_count,word_count,word_density,punctuation_count,true_sentiment
0,122075,B005LNA5VK,A2K3TU5BIS0RMF,Shelby,0,6,1,1341878400,tastes like grass,purchased health benefits unfortunately couldn...,112,-0.183333,0.483333,112,16,6.588235,0,Negative
1,315939,B000EDK3RS,A1SPVNAK9HBTAT,Special Education Teacher,1,5,1,1312243200,Contains MSG!,like taste dressing realized contains additive...,177,-0.500000,1.000000,177,23,7.375000,0,Negative
2,307427,B001D0IZ4O,A2C216Z6CXBCCA,Martha Drummond,1,1,1,1265846400,Hate it.,drink tasted like waterno chocolate taste stuc...,84,0.125000,0.758333,84,13,6.000000,0,Negative
3,502262,B004H4LJ8W,A2HWR20S8OKDG,Cali_Girl_In_LA,1,1,1,1347494400,Tastes too sweet,tastes sweet compared brands tried ended eatin...,84,0.275000,0.425000,84,13,6.000000,0,Negative
4,214099,B000TOB8QW,A1XCM5HHAWFFFC,Sharon A. Carroll,0,0,1,1337817600,Strange taste,ordered jars calorie free alfredo sauce thinki...,171,0.075000,0.575000,171,25,6.576923,0,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,413,B00061KYVI,A3JE18D4XD2PYF,pattymelt,0,0,5,1282521600,best roast ever,best way cook roast local grocery went buisnes...,117,0.466667,0.233333,117,18,6.157895,0,Positive
9996,20902,B001EPPCFS,AEO1L073HMSPQ,"Marshall's Sauces ""Marshall""",0,0,5,1253577600,Excellent,excellent product slightly different anything ...,638,0.282619,0.515357,638,94,6.715789,0,Positive
9997,76401,B001HTKOWC,A2IPHGXIAYZNJK,TG,2,2,5,1219622400,Wake Up Call,wife enjoy pot green chai tea morning great fl...,157,0.059028,0.443056,157,25,6.038462,0,Positive
9998,27857,B000SP1CWW,A2OYGD3W1AZEB9,Speed,0,0,5,1223251200,Great treat!,yorkie loves stuff vet gave first visit loved ...,69,0.475000,0.566667,69,12,5.307692,0,Positive


In [9]:
data["true_sentiment"].value_counts()

,count
true_sentiment,
Negative,4000
Positive,4000
Neutral,2000


In [10]:
def clean_review(text):
    text = text.lower()  # Lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r"\@w+|\#", '', text)
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = re.sub(r"\s+", ' ', text).strip()
    text = re.sub(r'<.*?>', ' ', text)
    for word in stop_words:
        text = text.replace(rf'\b{word}\b', '')
    return text


data["Text+Summary"] = data["Summary"] + " " + data["Text"]
data.dropna(subset=['Text+Summary'], inplace=True)
data["Text+Summary"] = data["Text+Summary"].apply(clean_review)

In [11]:
lemmatizer = WordNetLemmatizer()

num_lemmas = []
num_tokens = []

def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    num_lemmas.append(len(lemmas))
    return ' '.join(lemmas)

data["Text+Summary"] = data["Text+Summary"].apply(lemmatize_text)

In [12]:
sum(num_lemmas) / len(num_lemmas)

47.75237523752375

In [13]:
data["num_sentiment"] = data["true_sentiment"].map({"Positive": 2, "Neutral": 1, "Negative": 0})

In [14]:
data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,length,polarity,subjectivity,char_count,word_count,word_density,punctuation_count,true_sentiment,Text+Summary,num_sentiment
0,122075,B005LNA5VK,A2K3TU5BIS0RMF,Shelby,0,6,1,1341878400,tastes like grass,purchased health benefits unfortunately couldn...,112,-0.183333,0.483333,112,16,6.588235,0,Negative,taste like grass purchased health benefit unfo...,0
1,315939,B000EDK3RS,A1SPVNAK9HBTAT,Special Education Teacher,1,5,1,1312243200,Contains MSG!,like taste dressing realized contains additive...,177,-0.500000,1.000000,177,23,7.375000,0,Negative,contains msg like taste dressing realized cont...,0
2,307427,B001D0IZ4O,A2C216Z6CXBCCA,Martha Drummond,1,1,1,1265846400,Hate it.,drink tasted like waterno chocolate taste stuc...,84,0.125000,0.758333,84,13,6.000000,0,Negative,hate it drink tasted like waterno chocolate ta...,0
3,502262,B004H4LJ8W,A2HWR20S8OKDG,Cali_Girl_In_LA,1,1,1,1347494400,Tastes too sweet,tastes sweet compared brands tried ended eatin...,84,0.275000,0.425000,84,13,6.000000,0,Negative,taste too sweet taste sweet compared brand tri...,0
4,214099,B000TOB8QW,A1XCM5HHAWFFFC,Sharon A. Carroll,0,0,1,1337817600,Strange taste,ordered jars calorie free alfredo sauce thinki...,171,0.075000,0.575000,171,25,6.576923,0,Negative,strange taste ordered jar calorie free alfredo...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,413,B00061KYVI,A3JE18D4XD2PYF,pattymelt,0,0,5,1282521600,best roast ever,best way cook roast local grocery went buisnes...,117,0.466667,0.233333,117,18,6.157895,0,Positive,best roast ever best way cook roast local groc...,2
9996,20902,B001EPPCFS,AEO1L073HMSPQ,"Marshall's Sauces ""Marshall""",0,0,5,1253577600,Excellent,excellent product slightly different anything ...,638,0.282619,0.515357,638,94,6.715789,0,Positive,excellent excellent product slightly different...,2
9997,76401,B001HTKOWC,A2IPHGXIAYZNJK,TG,2,2,5,1219622400,Wake Up Call,wife enjoy pot green chai tea morning great fl...,157,0.059028,0.443056,157,25,6.038462,0,Positive,wake up call wife enjoy pot green chai tea mor...,2
9998,27857,B000SP1CWW,A2OYGD3W1AZEB9,Speed,0,0,5,1223251200,Great treat!,yorkie loves stuff vet gave first visit loved ...,69,0.475000,0.566667,69,12,5.307692,0,Positive,great treat yorkie love stuff vet gave first v...,2


# Train model

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words="english", max_features=5000) # Limit vocab size
# X = vectorizer.fit_transform(data["Text+Summary"])  # `reviews` is a list of strings (your Amazon review texts)

In [21]:
from sklearn.model_selection import train_test_split

X = data[["Text+Summary"]]
y = data["num_sentiment"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [22]:
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train)
y_test_enc = label_encoder.transform(y_test)

In [23]:
vectorizer = CountVectorizer(stop_words='english', max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec  = vectorizer.transform(X_test)

In [24]:
X_train_tensor = torch.tensor(X_train_vec.toarray(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_enc, dtype=torch.long)

X_test_tensor = torch.tensor(X_test_vec.toarray(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_enc, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32)

AssertionError: Size mismatch between tensors

In [25]:
import torch.nn as nn

class BoWClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [26]:
model = BoWClassifier(input_dim=5000, hidden_dim=128, output_dim=3)

In [27]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# model.train()
# for epoch in range(5):
#     for inputs, targets in train_loader:
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, targets)
#         loss.backward()
#         optimizer.step()
#     print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

for epoch in range(5):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

NameError: name 'train_loader' is not defined

In [ ]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, preds = torch.max(outputs, dim=1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

print(confusion_matrix(all_labels, all_preds))
target_names = [str(c) for c in label_encoder.classes_]
print(classification_report(all_labels, all_preds, target_names=target_names))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Confusion matrix
print("Confusion Matrix:")
print(cm := confusion_matrix(all_labels, all_preds))

# Classification report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["negative", "neutral", "positive"]))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Optional: define class labels
class_names = ["negative", "neutral", "positive"]

# Plot using seaborn (nicer visuals)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names,annot_kws={"size": 16})
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()